# Show Overview

In [ ]:
!pip install -U kaleido

In [ ]:
# Default
import glob
import os
from tqdm import tqdm
from datetime import datetime

# Data Hundling
import pandas as pd

# Visualize
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo
from IPython.display import Image
import kaleido

In [ ]:
DISTRICT_PATH = "../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv"
PRODUCTS_PATH = "../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv"
ENGAGEMENT_DIR = "../input/learnplatform-covid19-impact-on-digital-learning/engagement_data"

In [ ]:
engage_list = []
for file in tqdm(os.listdir(ENGAGEMENT_DIR)):
    path = os.path.join(ENGAGEMENT_DIR, file)
    engage_tmp_df = pd.read_csv(path)
    engage_tmp_df["district_id"] = file.split(".")[0]
    engage_list.append(engage_tmp_df)
engage_df = pd.concat(engage_list)

In [ ]:
district_df = pd.read_csv(DISTRICT_PATH)
district_df["district_id"] = district_df["district_id"].astype(str)

products_df = pd.read_csv(PRODUCTS_PATH)
products_df.columns = ["lp_id", "url", "product_name", "provider_company_name", "sector", "pef"]
products_df["lp_id"] = products_df["lp_id"].astype(int).astype(str)

engage_df = engage_df[engage_df["lp_id"].notnull()].reset_index(drop=True)
engage_df["lp_id"] = engage_df["lp_id"].astype(int).astype(str)
engage_df = engage_df.merge(district_df, on=["district_id"], how="left")
engage_df = engage_df.merge(products_df, on=["lp_id"], how="left")

engage_df["pct_access"].fillna(0, inplace=True)
engage_df["engagement_index"].fillna(0, inplace=True)
engage_df["product_name"].fillna("NAN", inplace=True)

In [ ]:
def piv(piv_col, val_col, df=engage_df, rolling=False, title=None, facet=False):
    if facet:
        pivs = (
            df
            .pivot_table(
                index=["time", facet], 
                columns=piv_col, 
                values=val_col, 
                aggfunc="mean"
            )
        )
    else:
        pivs = (
            df
            .pivot_table(
                index="time", 
                columns=piv_col, 
                values=val_col, 
                aggfunc="mean"
            )
        ) 
    
    if rolling:
        pivs = pivs.rolling(rolling).mean()

    if facet:
        return px.line(
            pivs.reset_index(), 
            x="time", 
            y=pivs.columns,
            facet_col=facet,
            facet_col_wrap=1,
            title=title
        )
    else:
        return px.line(
            pivs.reset_index(), 
            x="time", 
            y=pivs.columns,
            title=title
        )

# State by Locale

In [ ]:
state_counter = district_df.groupby(["state", "locale"])["district_id"].count().reset_index()
fig = px.bar(state_counter, x="state", y="district_id", color="locale")
Image(fig.to_image(format="png", scale=10))

There seems to be a difference in the regions that are able to obtain them in each state.
Note that each state has different policies/calendars, etc.

# Pacentage Access By Locale

In [ ]:
fig = piv(
    "locale", 
    "pct_access", 
    rolling=7,
    title="pct_access_all_product_average_mean_by_locale(7 days average)"
)

fig.add_vline(
    x=datetime(2020,3,19).timestamp() * 1000, 
    line_width=3, 
    line_dash="dash", 
    line_color="black", 
    annotation_text="First Lockdown(CA, etc)"
)
fig.add_vline(
    x=datetime(2020,6,22).timestamp() * 1000, 
    line_width=3, 
    line_dash="dash", 
    line_color="black", 
    annotation_text="School End(2019-2020)"
)

fig.add_vline(
    x=datetime(2020,9,8).timestamp() * 1000, 
    line_width=3, 
    line_dash="dash", 
    line_color="black", 
    annotation_text="School Start(2020-2021)",
    annotation_yclick=3
)

# Image(fig.to_image(format="png", scale=10))
fig.show()

* California Issues Statewide Stay-at-Home Order March 19 [https://www.ajmc.com/view/a-timeline-of-covid19-developments-in-2020]
* The beginning and end of the school year is a guide. The actual number varies from state to state/school to school.

### Before Covid-19

The average percentage of access by each locale is Rural>Town>Suburb>City in before Corona, although it should be noted that there are some products for which we have not been able to obtain data. Although demographics need to be confirmed, it is possible that the environment was easier to maintain in Rural/Town areas, which have fewer people, than in city areas, which have more people.

### Lockdown

After the lockdown, there was an instantaneous decrease in the access ratio in all regions, but in Rural/Suburb, it quickly returned to the same level as before the corona. In the Town, although the access rate quickly returned to the pre-Corona level, the access rate rapidly decreased around the beginning of May, earlier than in other regions, indicating that the early vacation period was a good time to take a break. In Town, the access rate quickly returned to the pre-Corona level, but from early May, the access rate decreased more rapidly than in other areas, suggesting that people are taking an early vacation.

### After start school 2020-2021

In Rural/Suburb/City, the access ratio has been higher since the start of the 2021 school year than before Corona, indicating a trend of progress in the access environment. However, in the Town, the access ratio has decreased compared to the pre-Corona period, and a deeper investigation of the cause is needed.

# Deep Dive in Utah

Since many of the measures will be implemented on a state-by-state basis, we will start by digging deeper into the data in Utah, where data exists for all locales.

In [ ]:
fig = piv(
    "locale", 
    "pct_access", 
    df=engage_df[engage_df["state"]=="Utah"],
    rolling=7,
    title="Utah_pct_access_all_product_average_mean_by_locale(7 days average)"
)

fig.add_vline(
    x=datetime(2020,3,16).timestamp() * 1000, 
    line_width=3, 
    line_dash="dash", 
    line_color="black", 
    annotation_text="School soft closure"
)

fig.add_vline(
    x=datetime(2020,6,1).timestamp() * 1000, 
    line_width=3, 
    line_dash="dash", 
    line_color="black", 
    annotation_text="School End(2019-2020)"
)

# https://www.schools.utah.gov/file/f27e32f6-d5ab-4886-bfda-dfa7f951e878
fig.add_vline(
    x=datetime(2020,8,17).timestamp() * 1000, 
    line_width=3, 
    line_dash="dash", 
    line_color="black", 
    annotation_text="School Start(2020-2021)"
)

fig.show()

### Before Covid-19

No significant change from the overall trend, but there was a slight upward trend in the City from early February.

### Lockdown(Soft Clousuer)

In Suburb/Town, access is temporarily lower, but then returns to the same level as before Corona.
On the other hand, in Rural/City, it will end FY2019 at a low level.

### After start school 2020-2021

Return to almost the same level as before the corona in all locales.

## Parcentage Access by Product/Locale

* LC(Learning & Curriculum)
    * Digital Learning Platforms,
    * Content Creation & Curation
    * Courseware & Textbooks
    * Career Planning & Job Search
    * Online Course Providers & Technical Skills Development
    * Study Tools
        * Test Prep & Study Skills
        * Q&A
        * Tutoring
        * Other
    * Sites, Resources & Reference
        * Games & Simulations
        * Streaming Services
        * Encyclopedia
        * Thesaurus & Dictionary
        * Learning Materials & Supplies
        * Other
* CM(Classroom Management)
    * Virtual Classroom
        * Video Conferencing & Screen Sharing
    * Classroom Engagement & Instruction
        * Classroom Management
        * Communication & Messaging
        * Assessment & Classroom Response
    * Teacher Resources
        * Professional Learning
        * Grading & Attendance
        * Lesson Planning
* SDO(School & District Operations)
    * Human Resources
    * Large-Scale & Standardized Testing
    * Admissions, Enrollment & Rostering
    * Environmental, Health & Safety (EHS) Compliance
    * Learning Management Systems (LMS)
    * School Management Software
        * SSO
        * Mobile Device Management
    * Data, Analytics & Reporting
        * Student Information Systems (SIS)
        * Site Hosting & Data Warehousing
        * Other
    * Other
* Other

In [ ]:
pef_list = [
    "LC - Digital Learning Platforms",
    "LC - Content Creation & Curation",
    "LC - Courseware & Textbooks", 
    "LC - Career Planning & Job Search",
    "LC - Online Course Providers & Technical Skills Development",
    "LC - Sites, Resources & Reference - Games & Simulations",
    "LC - Sites, Resources & Reference - Streaming Services",
    "LC - Sites, Resources & Reference - Digital Collection & Repository",
    "LC - Sites, Resources & Reference - Encyclopedia",
    "LC - Sites, Resources & Reference - Thesaurus & Dictionary",
    "LC - Sites, Resources & References - Learning Materials & Supplies",
    "LC - Sites, Resources & Reference",
    "LC - Study Tools - Test Prep & Study Skills",
    "LC - Study Tools - Q&A",
    "LC - Study Tools - Tutoring",
    "LC - Study Tools", 
    "CM - Virtual Classroom - Video Conferencing & Screen Sharing",
    "CM - Classroom Engagement & Instruction - Classroom Management",
    "CM - Classroom Engagement & Instruction - Communication & Messaging",
    "CM - Classroom Engagement & Instruction - Assessment & Classroom Response",
    "CM - Teacher Resources - Professional Learning",
    "CM - Teacher Resources - Grading & Attendance",
    "CM - Teacher Resources - Lesson Planning",
    "SDO - Human Resources",
    "SDO - Large-Scale & Standardized Testing",
    "SDO - Admissions, Enrollment & Rostering",
    "SDO - Environmental, Health & Safety (EHS) Compliance",
    "SDO - Learning Management Systems (LMS)",
    "SDO - School Management Software - SSO",
    "SDO - School Management Software - Mobile Device Management",
    "SDO - Data, Analytics & Reporting", 
    "SDO - Data, Analytics & Reporting - Student Information Systems (SIS)",
    "SDO - Data, Analytics & Reporting - Site Hosting & Data Warehousing",
    "SDO - Other",
    "LC/CM/SDO - Other",
]

In [ ]:
for pef in pef_list:
    fig = piv(
        "product_name", 
        "pct_access", 
        df=engage_df[
            (engage_df["state"]=="Utah")
            &
            (engage_df["pef"]==pef)
        ],
        rolling=7,
        facet="locale",
        title=pef
    )

    display(Image(fig.to_image(format="png", scale=10)))

In [ ]:
for pef in pef_list:
    fig = piv(
        "product_name", 
        "engagement_index", 
        df=engage_df[
            (engage_df["state"]=="Utah")
            &
            (engage_df["pef"]==pef)
        ],
        rolling=7,
        facet="locale",
        title=pef
    )

    display(Image(fig.to_image(format="png", scale=10)))